# Preprocesamiento Sparse

**Autor**: Arturo Sánchez Palacio

Basado en: https://github.com/lazyprogrammer

**Fecha de última revisión: 18/I/2020**

Para la utilización de las matrices Sparse (parte del módulo Numpy) en los Autoencoders se requiere un preprocesamiento especial. Procedemos a exponerlo:

Como siempre comenzamos instalando los módulos que necesitamos e importándolos:

In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

Cargamos los datos:

In [2]:
df = pd.read_csv('./data/preprocessed_rating.csv')

Calculamos el número total de usuarios y películas:

In [3]:
N = df.userId.max() + 1
M = df.movie_idx.max() + 1

Como es habitual en los procesos de Machine Learning dividimos en entrenamiento y test. En este caso fijamos el tamaño de entrenamiento al 80%:

In [4]:
df = shuffle(df)
umbral = int(0.8*len(df))
df_train = df.iloc[:umbral]
df_test = df.iloc[umbral:]

Iniciamos una matriz Sparse de N filas y M columnas:

In [5]:
from scipy.sparse import lil_matrix
A = lil_matrix((N, M))

In [6]:
A

<138493x26744 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in LInked List format>

Comenzamos actualizando el conjunto de entrenamiento:

#### To do. Definimos una función ` update_train` para actualizar el conjunto de entrenamiento. Básicamente toma una fila del dataframe y coloca el rating en el espacio usuario, película de la matriz. 

(__Ejemplo:__ Usuario 1 película: 10 rating en la fila 1 columna 10)

In [7]:
df_train.head()

,userId,movieId,rating,movie_idx
17812199,123155,5445,4.0,5351
16856283,116555,6764,1.5,6657
5872498,40426,2710,1.5,2626
936523,6281,41997,5.0,10745
8045358,55418,6942,4.5,6833


In [8]:
#def update_train(df):
#    for index, row in df.iterrows():
#       A[int(row["userId"]), int(row["movie_idx"])] = row["rating"]

count = 0
def update_train(fila):
    global count
    count += 1
    if count % 100000 == 0:
        print("Procesado: %.3f" % (float(count)/umbral))
    i = int(fila.userId)
    j = int(fila.movie_idx)
    A[i, j] = fila.rating
        
        
    

#### To do. Aplicar la función al conjunto de entrenamiento:

In [9]:
df_train.apply(update_train, axis=1)

Procesado: 0.006
Procesado: 0.012
Procesado: 0.019
Procesado: 0.025
Procesado: 0.031
Procesado: 0.037
Procesado: 0.044
Procesado: 0.050
Procesado: 0.056
Procesado: 0.062
Procesado: 0.069
Procesado: 0.075
Procesado: 0.081
Procesado: 0.087
Procesado: 0.094
Procesado: 0.100
Procesado: 0.106
Procesado: 0.112
Procesado: 0.119
Procesado: 0.125
Procesado: 0.131
Procesado: 0.137
Procesado: 0.144
Procesado: 0.150
Procesado: 0.156
Procesado: 0.162
Procesado: 0.169
Procesado: 0.175
Procesado: 0.181
Procesado: 0.187
Procesado: 0.194
Procesado: 0.200
Procesado: 0.206
Procesado: 0.212
Procesado: 0.219
Procesado: 0.225
Procesado: 0.231
Procesado: 0.237
Procesado: 0.244
Procesado: 0.250
Procesado: 0.256
Procesado: 0.262
Procesado: 0.269
Procesado: 0.275
Procesado: 0.281
Procesado: 0.287
Procesado: 0.294
Procesado: 0.300
Procesado: 0.306
Procesado: 0.312
Procesado: 0.319
Procesado: 0.325
Procesado: 0.331
Procesado: 0.337
Procesado: 0.344
Procesado: 0.350
Procesado: 0.356
Procesado: 0.362
Procesado: 0.3

17812199    None
16856283    None
5872498     None
936523      None
8045358     None
14854626    None
13167421    None
1483599     None
18120847    None
17395408    None
1116269     None
15175710    None
16639825    None
11663732    None
1177483     None
8849612     None
433813      None
12304733    None
12422705    None
10627974    None
3945548     None
14395113    None
19479079    None
9874062     None
13207365    None
18020258    None
16403665    None
16599660    None
18279317    None
3182734     None
            ... 
13127206    None
15148318    None
6661191     None
5948200     None
7330605     None
10792586    None
5188003     None
1111587     None
11416222    None
6819512     None
6352092     None
17035538    None
10871673    None
16842198    None
8901748     None
4953814     None
7427132     None
15682442    None
2670122     None
6844323     None
17630636    None
132771      None
9005214     None
8349097     None
9082876     None
17724836    None
4370710     None
17682732    No

Una vez que la matriz ha sido rellenada usamos una máscara para saber qué valores existen y cuales no:

__Nota.__ `toscr()` devuelve una copia de la matriz en formato Compressed Sparse Row. El formato `lil` es más eficiente para añadir valores y el formato `scr` ocupa menos memoria en disco.

In [10]:
A = A.tocsr()
mask = (A > 0)

Ya tenemos nuestra matriz sparse. La almacenamos en un archivo `.npz`:

In [11]:
from scipy.sparse import save_npz
save_npz("./data/Atrain.npz", A)

Realizamos el mismo proceso para test:

In [12]:
A_test = lil_matrix((N, M))
print("Actualizando test")
count = 0 #necesario volver a ponerlo a 0
def update_test(fila):
    global count
    count += 1
    if count % 100000 == 0:
        print("Procesado: %.3f" % (float(count)/len(df_test))) #menos cantidad luego es mucho más rápida

    i = int(fila.userId)
    j = int(fila.movie_idx)
    A_test[i,j] = fila.rating
df_test.apply(update_test, axis=1)
A_test = A_test.tocsr()
mask_test = (A_test > 0)
save_npz("./data/Atest.npz", A_test)

Actualizando test
Procesado: 0.025
Procesado: 0.050
Procesado: 0.075
Procesado: 0.100
Procesado: 0.125
Procesado: 0.150
Procesado: 0.175
Procesado: 0.200
Procesado: 0.225
Procesado: 0.250
Procesado: 0.275
Procesado: 0.300
Procesado: 0.325
Procesado: 0.350
Procesado: 0.375
Procesado: 0.400
Procesado: 0.425
Procesado: 0.450
Procesado: 0.475
Procesado: 0.500
Procesado: 0.525
Procesado: 0.550
Procesado: 0.575
Procesado: 0.600
Procesado: 0.625
Procesado: 0.650
Procesado: 0.675
Procesado: 0.700
Procesado: 0.725
Procesado: 0.750
Procesado: 0.775
Procesado: 0.800
Procesado: 0.825
Procesado: 0.850
Procesado: 0.875
Procesado: 0.900
Procesado: 0.925
Procesado: 0.950
Procesado: 0.975
Procesado: 1.000


Con esto hemos terminado el preprocesamiento y hemos almacenado en sendos archivos `Atrain.npz` y `Atest.npz` las matrices sparse de entrenamiento y test.